In [1]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import os
from time import sleep

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup

In [2]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# get the chromedriver path
path = os.getcwd()+'/chromedriver'
options = Options()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--single-process')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument("window-size=1920,1080")
# options.add_argument("user-agent=User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36")
# options.add_argument("referer=Referer: https://iprice.co.id/")
driver = webdriver.Chrome(options=options)

In [4]:
header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'referer':'https://iprice.co.id/'
}

In [5]:
url = 'https://iprice.co.id/ponsel-tablet/smartphone/?page='

In [6]:
def get_link_name(url):
    test = requests.get(url = url, headers = header)
    bsobj = BeautifulSoup(test.content,'html.parser')
    Website = []
    Nama_Smartphone = []
    for item in bsobj.findAll('div', {'data-vars-cgt':'click_comparable_product'}):
        Website.append(item.a['href'])
        data_vars = item['data-vars-lb'].split('|')[0]
        Nama_Smartphone.append(data_vars)
    for item in bsobj.findAll('a', {'data-vars-cgt':'click_comparable_product'}):
        Website.append(item['href'])
        try:
          data_vars = item['data-vars-lb'].split('|')[0]
          Nama_Smartphone.append(data_vars)
        except:
          continue
    if len(Website)>59:
        Website = Website[:59]
    if len(Nama_Smartphone)>59:
        Nama_Smartphone = Nama_Smartphone[:59]

    df = pd.DataFrame(Nama_Smartphone, columns = ['Smartphone Title'])
    df["Website"] = Website

    return df

In [7]:
def click_link(url):
    detail = requests.get(url = url, headers = header)
    bsobj = BeautifulSoup(detail.content,'html.parser')
    
    variant_list = []
    for item in bsobj.findAll('a', {'data-vars-cgt':'click_product_characteristic'}):
        variant_list.append(item['href'])
        
    #for item in bsobj.findAll('a', {'class':'lq rg p1 ek aZ a- mD tN lY w4 fk su mc'}):
     #   variant_list.append(item['href'])
    
    return variant_list

In [8]:
def get_detail(url):
    #Run the webdriver
    driver.get(url)
    #driver.maximize_window()
    driver.execute_script("window.scrollTo(0, 600)")
    sleep(2)

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="excellent-offers-tab"]'))).click()

    sleep(2)
    soup = driver.page_source
    bsobj_detail = BeautifulSoup(soup,'html.parser')
    
    # Get the offername
    Offername = []
    for item in bsobj_detail.findAll('div', {'class':'zE dn a en'}):
        Offername.append(item.text.strip())
    if len(Offername)>20:
        Offername = Offername[:20]

    Offername = list(filter(None, Offername))
       
    # Scrape the Price
    Price = []
    for item in bsobj_detail.findAll('div', {'class':'a_ aw f24-l gW'}):
        Price.append(item.text.strip())

    if len(Price)>20:
        Price = Price[:20]

    Price = list(filter(None, Price))
        
    
    # Get the platform
    Platform = []
    SellerRating = []
    SellerName = []

    for item in bsobj_detail.findAll('div', {'data-vars-action':'pc-2ndhand'}):
        line = item["data-vars-merchant"]
        Platform.append(line.split('|')[0])
        SellerName.append(line.split('|')[2].replace('sellerName:',''))
        SellerRating.append(line.split('|')[3].replace('sellerRating:',''))

    Platform = Platform[1:21]
    SellerRating = SellerRating[1:21]
    SellerName = SellerName[1:21]

    Platform = list(filter(None, Platform))

    Platform = [ x for x in Platform if '{{merchant.url}}' not in x ]

    SellerName = [ x for x in SellerName if '{{seller.name}}' not in x ]

    SellerRating = [ x for x in SellerRating if '{{seller.rating}}' not in x ]
    
    # Processing to match the same length
    la,lb,lc,ld,le = len(Offername),len(Price),len(Platform),len(SellerRating),len(SellerName)
    min_len = min(la,lb,lc,ld,le)
    
    if len(Offername)>min_len:
        Offername = Offername[:min_len - len(Offername)]
    
    if len(Price)>min_len:
        Price = Price[:min_len - len(Price)]

    if len(Platform)>min_len:
        Platform = Platform[:min_len - len(Platform)]

    if len(SellerRating)>min_len:
        SellerRating = SellerRating[:min_len - len(SellerRating)]

    if len(SellerName)>min_len:
        SellerName = SellerName[:min_len - len(SellerName)]
    
    # Other Specs
    try:
        Warna = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Warna" in tag.text)[1].text.replace("Warna",'').strip()
    except:
        Warna = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Warna" in tag.text)[0].text.replace("Warna",'').strip()
    try:
      ROM = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Penyimpanan" in tag.text)[0].text.replace("Penyimpanan", '').strip()
    except:
      ROM = None
    try:
      RAM = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "RAM" in tag.text)[0].text.replace("RAM", '').strip()
    except:
      RAM = None
    try:
        Harga_resmi = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Harga Resmi" in tag.text)[0].text.replace("Harga Resmi", '').strip()
    except:
        Harga_resmi = None
    Kamera = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Kamera Utama" in tag.text)[0].text.replace("Kamera Utama", '').strip()
    
    try:
      Layar = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Layar" in tag.text)[0].text.replace("Layar",'').strip()
    except:
      Layar = None
    
    
    # Make the table
    data = {
        'Offername':Offername,
        'Price_Seller':Price,
        'Platform':Platform,
        'SellerRating':SellerRating,
        'SellerName':SellerName,
    }
    
    df = pd.DataFrame(data)
    df["Colour"] = Warna
    df["ROM"] = ROM
    df["RAM"] = RAM
    df["Price Official"] = Harga_resmi
    df["Camera"] = Kamera
    df["Screen Size"]=Layar
    
    return df

In [9]:
def get_detail2(url):
    #Run the webdriver
    driver.get(url)
    #driver.maximize_window()
    driver.execute_script("window.scrollTo(0, 600)")
    sleep(2)
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="unknown-offers-tab-in-default"]'))).click()
        
    sleep(2)
    soup = driver.page_source
    bsobj_detail = BeautifulSoup(soup,'html.parser')
    
    # Get the offername
    Offername = []
    for item in bsobj_detail.findAll('div', {'class':'b8 l4 rL F mD ve aZ ph A a0 eC l7'}):
        now = item.findAll('p',{'class':'qA b a dG rM'})
        for each in now:
            Offername.append(each.text.strip())

    if len(Offername)>20:
        Offername = Offername[:20]

    Offername = list(filter(None, Offername))
    
    # Scrape the Price
    Price = []
    for item in bsobj_detail.findAll('div', {'class':'b8 l4 rL F mD ve aZ ph A a0 eC l7'}):
        now = item.findAll('div',{'class':'e7 a1 b gW tr bT dn bR'})
        for each in now:
            Price.append(each.text.strip())

    if len(Price)>20:
        Price = Price[:20]

    Price = list(filter(None, Price))       
    
    # Get the platform
    Platform = []
    SellerRating = []
    SellerName = []

    for item in bsobj_detail.findAll('span', {'data-vars-action':'pc-2ndhand'}):
        line = item["data-vars-merchant"]
        Platform.append(line.split('|')[0])
        SellerName.append(line.split('|')[2].replace('sellerName:',''))
        SellerRating.append(line.split('|')[3].replace('sellerRating:',''))

    Platform = Platform[:21]
    SellerRating = SellerRating[:21]
    SellerName = SellerName[:21]

    Platform = list(filter(None, Platform))

    Platform = [ x for x in Platform if '{{merchant.url}}' not in x ]

    SellerName = [ x for x in SellerName if '{{seller.name}}' not in x ]

    SellerRating = [ x for x in SellerRating if '{{seller.rating}}' not in x ]
      
    # Processing to match the same length
    la,lb,lc,ld,le = len(Offername),len(Price),len(Platform),len(SellerRating),len(SellerName)
    min_len = min(la,lb,lc,ld,le)
    
    if len(Offername)>min_len:
        Offername = Offername[:min_len - len(Offername)]
    
    if len(Price)>min_len:
        Price = Price[:min_len - len(Price)]

    if len(Platform)>min_len:
        Platform = Platform[:min_len - len(Platform)]

    if len(SellerRating)>min_len:
        SellerRating = SellerRating[:min_len - len(SellerRating)]

    if len(SellerName)>min_len:
        SellerName = SellerName[:min_len - len(SellerName)]
    
    # Other Specs
    try:
        Warna = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Warna" in tag.text)[1].text.replace("Warna",'').strip()
    except:
        Warna = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Warna" in tag.text)[0].text.replace("Warna",'').strip()
    try:
      ROM = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Penyimpanan" in tag.text)[0].text.replace("Penyimpanan", '').strip()
    except:
      ROM = None
    try:
      RAM = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "RAM" in tag.text)[0].text.replace("RAM", '').strip()
    except:
      RAM = None
    
    try:
        Harga_resmi = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Harga Resmi" in tag.text)[0].text.replace("Harga Resmi", '').strip()
    except:
        Harga_resmi = None
    Kamera = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Kamera Utama" in tag.text)[0].text.replace("Kamera Utama", '').strip()
    try:
      Layar = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Layar" in tag.text)[0].text.replace("Layar",'').strip()
    except:
      Layar = None
    
    
    # Make the table
    data = {
        'Offername':Offername,
        'Price_Seller':Price,
        'Platform':Platform,
        'SellerRating':SellerRating,
        'SellerName':SellerName,
    }
    
    df = pd.DataFrame(data)
    df["Colour"] = Warna
    df["ROM"] = ROM
    df["RAM"] = RAM
    df["Price Official"] = Harga_resmi
    df["Camera"] = Kamera
    df["Screen Size"]=Layar
    
    return df

In [10]:
df_detail = pd.DataFrame()

In [22]:
df_general = pd.DataFrame()

In [23]:
for i in range(21,61):
    url = 'https://iprice.co.id/ponsel-tablet/smartphone/?page='+str(i)
    df_gen = get_link_name(url)
    df_general = pd.concat([df_general, df_gen], ignore_index=True)
    print(i)

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [24]:
df_general

,Smartphone Title,Website
0,Huawei P10 Plus,https://iprice.co.id/harga/huawei-p10-plus/
1,Oppo R9s Plus,https://iprice.co.id/harga/oppo-r9s-plus/
2,ASUS ZenFone C,https://iprice.co.id/harga/asus-zenfone-c/
3,Huawei Mate Xs,https://iprice.co.id/harga/huawei-mate-xs/
4,Google Pixel,https://iprice.co.id/harga/google-pixel/
...,...,...
466,Realme C11 32GB 3GB Pepper Grey,https://iprice.co.id/harga/realme-c11-32gb-3gb...
467,Apple iPhone 15 Pro Max 1TB Natural Titanium,https://iprice.co.id/harga/apple-iphone-15-pro...
468,Samsung Galaxy Z Fold4 512GB Burgundy,https://iprice.co.id/harga/samsung-galaxy-z-fo...
469,Xiaomi Mi Max 2 64GB Hitam,https://iprice.co.id/harga/xiaomi-mi-max-2-64g...


In [25]:
for i in tqdm(range(len(df_general))):
    link = df_general.iloc[i,1]
    try:
        try:
            get_detail(link)
        except:
            get_detail2(link)
    except:
        print("Failed"+link)
        continue

    variant = click_link(link)
    for detail_link in variant:
        try:
            try:
                dfs = get_detail(detail_link)
                dfs["Smartphone Title"] = df_general.iloc[i,0]
                dfs["Link"] = detail_link
                df_detail = pd.concat([df_detail, dfs], ignore_index=True)
            except:
                dfs = get_detail2(detail_link)
                dfs["Smartphone Title"] = df_general.iloc[i,0]
                dfs["Link"] = detail_link
                df_detail = pd.concat([df_detail, dfs], ignore_index=True)
        except:
            continue

    print(link)

  0%|▏                                                                               | 1/471 [00:16<2:10:48, 16.70s/it]

Failedhttps://iprice.co.id/harga/huawei-p10-plus/


  0%|▎                                                                               | 2/471 [00:41<2:49:01, 21.62s/it]

Failedhttps://iprice.co.id/harga/oppo-r9s-plus/


  1%|▌                                                                               | 3/471 [01:07<3:02:48, 23.44s/it]

...

https://iprice.co.id/harga/samsung-galaxy-a20s-32gb-hijau/


 82%|████████████████████████████████████████████████████████████████              | 387/471 [8:23:02<13:54,  9.94s/it]

https://iprice.co.id/harga/xiaomi-redmi-note-9-64gb-sunset-red/
Failedhttps://iprice.co.id/harga/apple-iphone-12-mini-256gb-hijau/
Failedhttps://iprice.co.id/harga/xiaomi-poco-x3-gt-256gb-stargaze-black/
Failedhttps://iprice.co.id/harga/apple-iphone-7-256gb-red/
Failedhttps://iprice.co.id/harga/samsung-galaxy-a04e-64gb-4gb-light-blue/
Failedhttps://iprice.co.id/harga/infinix-note-11-nfc-128gb-celestial-snow/
Failedhttps://iprice.co.id/harga/xiaomi-redmi-note-7-128gb-4gb-biru/
Failedhttps://iprice.co.id/harga/samsung-galaxy-s21-5g-256gb-phantom-pink/
Failedhttps://iprice.co.id/harga/nokia-c31-mint/
Failedhttps://iprice.co.id/harga/nokia-c31-mint/
Failedhttps://iprice.co.id/harga/realme-gt-master-edition-256gb-voyager-grey/
Failedhttps://iprice.co.id/harga/tecno-pova-neo-2-128gb-4gb-magma-orange/
Failedhttps://iprice.co.id/harga/samsung-galaxy-a80-emas/
Failedhttps://iprice.co.id/harga/infinix-hot-12-play-64gb-4gb-origin-blue/
Failedhttps://iprice.co.id/harga/oneplus-8t/
Failedhttps://ip

100%|██████████████████████████████████████████████████████████████████████████████| 471/471 [8:23:02<00:00, 64.08s/it]

Failedhttps://iprice.co.id/harga/xiaomi-redmi-note-5a-prime-64gb-emas/
Failedhttps://iprice.co.id/harga/tecno-pova-4-pro-128gb-uranolith-grey/
Failedhttps://iprice.co.id/harga/realme-c30s-64gb-4gb-stripe-black/
Failedhttps://iprice.co.id/harga/realme-c30s-64gb-4gb-stripe-black/
Failedhttps://iprice.co.id/harga/oppo-f11-128gb-4gb-marble-green/
Failedhttps://iprice.co.id/harga/tecno-pova-4-pro-128gb-flourite-blue/
Failedhttps://iprice.co.id/harga/vivo-y20s-128gb-6gb-obsidian-black/
Failedhttps://iprice.co.id/harga/vivo-v15-pro-topaz-blue/
Failedhttps://iprice.co.id/harga/alcatel-flash-plus-2/
Failedhttps://iprice.co.id/harga/samsung-galaxy-a22-5g-abu-abu/
Failedhttps://iprice.co.id/harga/oppo-a16e-32gb-3gb-putih/
Failedhttps://iprice.co.id/harga/huawei-honor-10-lite-64gb-4gb-sky-blue/
Failedhttps://iprice.co.id/harga/samsung-galaxy-a7-2018-64gb-hitam/
Failedhttps://iprice.co.id/harga/wiko-harry/
Failedhttps://iprice.co.id/harga/oppo-f11-128gb-4gb-marble-green/
Failedhttps://iprice.co.id/

In [28]:
df_detail

,Offername,Price_Seller,Platform,SellerRating,SellerName,Colour,ROM,RAM,Price Official,Camera,Screen Size,Smartphone Title,Link
0,Infinix Second Hot 30I Ram 8/128Gb (Blue) Fullset,Rp 1.140.000 Stok Habis,shopee,4.83,ANITA BATURAJA,Biru,None,None,None,50 + AIMP,"6.6""",Infinix Hot 30i,https://iprice.co.id/harga/infinix-hot-30i-128...
1,Infinix Hot 30I Ram 8/128Gb Garansi Resmi,Rp 1.196.000,shopee,4.96,ASTON CELLULER 48,Biru,None,None,None,50 + AIMP,"6.6""",Infinix Hot 30i,https://iprice.co.id/harga/infinix-hot-30i-128...
2,Infinix Hot 30I 8/128Gb Hp Second Seken Bekas ...,Rp 1.349.000 Stok Habis,shopee,4.88,Juraganhandphone_,Biru,None,None,None,50 + AIMP,"6.6""",Infinix Hot 30i,https://iprice.co.id/harga/infinix-hot-30i-128...
3,Infinix Hot 30I Ram 8Gb Internal 128Gb Handpho...,Rp 1.349.000 Stok Habis,shopee,4.84,mayphonecell,Biru,None,None,None,50 + AIMP,"6.6""",Infinix Hot 30i,https://iprice.co.id/harga/infinix-hot-30i-128...
4,Infinix Hot 30I 8/128Gb Second,Rp 1.350.000,shopee,4.84,ekostarcell,Biru,None,None,None,50 + AIMP,"6.6""",Infinix Hot 30i,https://iprice.co.id/harga/infinix-hot-30i-128...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17057,Xiaomi Redmi Note 9 Ram 4/64 Warna Hitam,Rp 1.700.000 Stok Habis,shopee,4.55,perdian69,Hitam,64GB,4GB,1.999.000,48 + 8 + 2 + 2MP,"6.53""",Xiaomi Redmi Note 9 64GB Sunset Red,https://iprice.co.id/harga/xiaomi-redmi-note-9...
17058,Xiaomi Redmi Note 9 6/128Gb & 4/64 Gb Bekas Se...,Rp 1.739.997 Stok Habis,shopee,4.81,Niephonecell,Hitam,64GB,4GB,1.999.000,48 + 8 + 2 + 2MP,"6.53""",Xiaomi Redmi Note 9 64GB Sunset Red,https://iprice.co.id/harga/xiaomi-redmi-note-9...
17059,"Xiaomi Redmi Note 9 6/128, 4/64 Lengkap",Rp 1.800.000,shopee,4.72,Dzikry Phone,Hitam,64GB,4GB,1.999.000,48 + 8 + 2 + 2MP,"6.53""",Xiaomi Redmi Note 9 64GB Sunset Red,https://iprice.co.id/harga/xiaomi-redmi-note-9...
17060,Xiaomi Note 9 Second,Rp 1.850.000,shopee,4.97,Amin_mart,Hitam,64GB,4GB,1.999.000,48 + 8 + 2 + 2MP,"6.53""",Xiaomi Redmi Note 9 64GB Sunset Red,https://iprice.co.id/harga/xiaomi-redmi-note-9...


In [29]:
df_detail.to_csv("Iprice_SecondhandPhone_27September2023_2.csv")

In [68]:
get_detail()

,Offername,Price_Seller,Platform,SellerRating,SellerName,Colour,ROM,RAM,Price Official,Camera


In [99]:
driver.quit()

In [12]:
click_link(df_general["Website"][1])

['https://iprice.co.id/harga/apple-iphone-13-pro-max-128gb-biru-sierra/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-256gb-abu-abu-grafit/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-512gb-abu-abu-grafit/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-1tb-perak/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-128gb-abu-abu-grafit/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-128gb-alpine-green/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-128gb-biru-sierra/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-128gb-emas/',
 'https://iprice.co.id/harga/apple-iphone-13-pro-max-128gb-perak/']

## Testing Driver

In [35]:
driver.get("https://iprice.co.id/harga/apple-iphone-12-pro-128gb-perak/")
#driver.maximize_window()
driver.execute_script("window.scrollTo(0, 600)")
sleep(2)

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="excellent-offers-tab"]'))).click()

sleep(2)
soup = driver.page_source
bsobj_detail = BeautifulSoup(soup,'html.parser')

In [36]:
# Get the offername
Offername = []
for item in bsobj_detail.findAll('div', {'class':'zE dn a en'}):
    Offername.append(item.text.strip())
if len(Offername)>20:
    Offername = Offername[:20]

Offername = list(filter(None, Offername))

# Scrape the Price
Price = []
for item in bsobj_detail.findAll('div', {'class':'a_ aw f24-l gW'}):
    Price.append(item.text.strip())

if len(Price)>20:
    Price = Price[:20]

Price = list(filter(None, Price))

In [37]:
Platform = []
SellerRating = []
SellerName = []

for item in bsobj_detail.findAll('div', {'data-vars-action':'pc-2ndhand'}):
    line = item["data-vars-merchant"]
    Platform.append(line.split('|')[0])
    SellerName.append(line.split('|')[2].replace('sellerName:',''))
    SellerRating.append(line.split('|')[3].replace('sellerRating:',''))

In [38]:
Platform

['shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 'shopee',
 '{{merchant.url}}']

In [39]:
SellerRating

['4.8',
 '4.9',
 '4.84',
 '4.84',
 '4.84',
 '4.77',
 '4.97',
 '4.97',
 '4.86',
 '4.86',
 '4.97',
 '4.91',
 '5',
 '4.9',
 '4.91',
 '4.95',
 '4.97',
 '4.97',
 '5',
 '4.86',
 '5',
 '4.95',
 '{{seller.rating}}']

In [40]:
SellerName

["{offer['seller']['name']}",
 "{offer['seller']['name']}",
 'Apple Neon Store',
 'Wish Phonestore',
 'Facai.iphone',
 'Cellular Shop 9k',
 'Padang Gadget Bukittinggi',
 'Padang Gadget Bukittinggi',
 'FEDOFONE SMARTPHONE',
 'FEDOFONE SMARTPHONE',
 'Mobile Phone Shop BEC',
 'Digicell\xa0Official',
 'iPro.Apple_Pati',
 'Newstore Bandung',
 'ekkystore_focus',
 'iphonecentergresik',
 'Padang Gadget ID',
 'Padang Gadget ID',
 'favorite_cellular',
 'Negri ponsel_',
 'HCSHOP*',
 'iphonecentergresik',
 '{{seller.name}}']

In [112]:
url = click_link('https://iprice.co.id/harga/samsung-galaxy-s22-ultra/#amp-x-autoplay=1')[1]

In [113]:
url

'https://iprice.co.id/harga/samsung-galaxy-s22-ultra-256gb-12gb-phantom-black/'

In [115]:
get_detail2('https://iprice.co.id/harga/xiaomi-redmi-note-11-128gb-4gb-graphite-gray/')

,Offername,Price_Seller,Platform,SellerRating,SellerName,Colour,ROM,RAM,Price Official,Camera
0,Xiaomi Redmi Note 11 Ram 6/128 + 4/128Gb Handp...,Rp 1.799.000,shopee,4.89,Android Zone_,Abu-abu,128GB,4GB,2.499.000,50 + 8MP
1,Xiaomi Redmi Note 11 Ram 6/128 + 4/128Gb Handp...,Rp 1.949.000,shopee,4.89,Android Zone_,Abu-abu,128GB,4GB,2.499.000,50 + 8MP
2,Xiaomi Ex Display/ Ex Demo Redmi Note 11 (4+12...,Rp 1.989.000,shopee,4.89,One world Mi Home,Abu-abu,128GB,4GB,2.499.000,50 + 8MP
3,Xiaomi Redmi Note 11 6/128 Baru Garansi Resmi,Rp 1.999.000,shopee,4.88,jimurape_store,Abu-abu,128GB,4GB,2.499.000,50 + 8MP
4,Xiaomi Redmi Note 11 4/128Gb & 6/128Gb Second ...,Rp 2.049.000,shopee,4.85,WE_STORE GADGET,Abu-abu,128GB,4GB,2.499.000,50 + 8MP
5,Xiaomi Redmi Note 11 - Second Original,Rp 2.200.000,shopee,4.84,hapemart gsk official partner,Abu-abu,128GB,4GB,2.499.000,50 + 8MP
6,Xiaomi Redmi Note 11 4/128 Dan 6/128,Rp 2.399.000,shopee,4.82,Boutiquee.official,Abu-abu,128GB,4GB,2.499.000,50 + 8MP


In [37]:
driver.get(url)

In [38]:
driver.maximize_window()

In [39]:
driver.execute_script("window.scrollTo(0, 800)")

In [41]:
try:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="excellent-offers-tab"]'))).click()
except:
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="unknown-offers-tab-in-default"]'))).click()

In [51]:
soup = driver.page_source
bsobj_detail = BeautifulSoup(soup,'html.parser')

In [43]:
# Get the offername
Offername = []
for item in bsobj_detail.findAll('div', {'class':'product-name dn bR a en'}):
    Offername.append(item.text.strip())
if len(Offername)>20:
    Offername = Offername[:20]

Offername = list(filter(None, Offername))

In [44]:
Offername

[]

In [62]:
# Scrape the Price
Price = []
for item in bsobj_detail.findAll('div', {'class':'price-2nd a_ aw gW'}):
    Price.append(item.text.strip())

if len(Price)>20:
    Price = Price[:20]
    
Price = list(filter(None, Price))

In [63]:
Price

[]

In [57]:
# Get the platform
Platform = []
SellerRating = []
SellerName = []

for item in bsobj_detail.findAll('div', {'data-vars-action':'pc-2ndhand'}):
    line = item["data-vars-merchant"]
    Platform.append(line.split('|')[0])
    SellerName.append(line.split('|')[2].replace('sellerName:',''))
    SellerRating.append(line.split('|')[3].replace('sellerRating:',''))

Platform = Platform[1:21]
SellerRating = SellerRating[1:21]
SellerName = SellerName[1:21]

Platform = list(filter(None, Platform))

Platform = [ x for x in Platform if '{{merchant.url}}' not in x ]

SellerName = [ x for x in SellerName if '{{seller.name}}' not in x ]

SellerRating = [ x for x in SellerRating if '{{seller.rating}}' not in x ]

In [58]:
Platform

[]

In [119]:
la,lb,lc,ld,le = len(Offername),len(Price),len(Platform),len(SellerRating),len(SellerName)
# now find the min
min_len = min(la,lb,lc,ld,le)

In [120]:
if len(Offername)>min_len:
    Offername = Offername[:min_len - len(Offername)]
    
if len(Price)>min_len:
    Price = Price[:min_len - len(Price)]
    
if len(Platform)>min_len:
    Platform = Platform[:min_len - len(Platform)]

if len(SellerRating)>min_len:
    SellerRating = SellerRating[:min_len - len(SellerRating)]

if len(SellerName)>min_len:
    SellerName = SellerName[:min_len - len(SellerName)]

In [121]:
data = {
        'Offername':Offername,
        'Price_Seller':Price,
        'Platform':Platform,
        'SellerRating':SellerRating,
        'SellerName':SellerName,
    }

In [122]:
# Other Specs
Warna = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Warna" in tag.text)[1].text.replace("Warna",'').strip()
ROM = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Penyimpanan" in tag.text)[0].text.replace("Penyimpanan", '').strip()
RAM = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "RAM" in tag.text)[0].text.replace("RAM", '').strip()
try:
    Harga_resmi = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Harga Resmi" in tag.text)[0].text.replace("Harga Resmi", '').strip()
except:
    Harga_resmi = None
Kamera = bsobj_detail.find_all(lambda tag: tag.name == "tr" and "Kamera Utama" in tag.text)[0].text.replace("Kamera Utama", '').strip()

In [123]:
df = pd.DataFrame(data)
df["Colour"] = Warna
df["ROM"] = ROM
df["RAM"] = RAM
df["Price Official"] = Harga_resmi
df["Camera"] = Kamera

## Testing Driver 2

In [72]:
url = click_link('https://iprice.co.id/harga/xiaomi-redmi-note-11/#amp-x-autoplay=1')[0]

In [73]:
url

'https://iprice.co.id/harga/xiaomi-redmi-note-11-128gb-4gb-graphite-gray/'

In [74]:
driver.get(url)

In [75]:
driver.maximize_window()

In [76]:
driver.execute_script("window.scrollTo(0, 600)")

In [77]:
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[@id="unknown-offers-tab-in-default"]'))).click()

In [80]:
soup = driver.page_source
bsobj_detail = BeautifulSoup(soup,'html.parser')

In [103]:
# Get the offername
Offername = []
for item in bsobj_detail.findAll('div', {'class':'b8 l4 rL F mD ve aZ ph A a0 eC l7'}):
    now = item.findAll('p',{'class':'qA b a dG rM'})
    for each in now:
        Offername.append(each.text.strip())
      
if len(Offername)>20:
    Offername = Offername[:20]

Offername = list(filter(None, Offername))

In [105]:
# Scrape the Price
Price = []
for item in bsobj_detail.findAll('div', {'class':'b8 l4 rL F mD ve aZ ph A a0 eC l7'}):
    now = item.findAll('div',{'class':'e7 a1 b gW tr bT dn bR'})
    for each in now:
        Price.append(each.text.strip())
        
if len(Price)>20:
    Price = Price[:20]
    
Price = list(filter(None, Price))

In [107]:
# Get the platform
Platform = []
SellerRating = []
SellerName = []

for item in bsobj_detail.findAll('span', {'data-vars-action':'pc-2ndhand'}):
    line = item["data-vars-merchant"]
    Platform.append(line.split('|')[0])
    SellerName.append(line.split('|')[2].replace('sellerName:',''))
    SellerRating.append(line.split('|')[3].replace('sellerRating:',''))

Platform = Platform[:21]
SellerRating = SellerRating[:21]
SellerName = SellerName[:21]

Platform = list(filter(None, Platform))

Platform = [ x for x in Platform if '{{merchant.url}}' not in x ]

SellerName = [ x for x in SellerName if '{{seller.name}}' not in x ]

SellerRating = [ x for x in SellerRating if '{{seller.rating}}' not in x ]